# Billboard Hot 100

## Import Dependencies 

In [1]:
#Importing dependencies for web scrapping
from bs4 import BeautifulSoup
import requests
import pandas as pd


## Web Scrapping -  Hot 100

In [2]:
from web_scraping import web_scraping

In [3]:
# running the web scrapping function
top_100_scrapped_df = web_scraping()

# Spotify Data Search

## Spotify for Developers - Using Spotify Library 

In [4]:
"""
Link to spotipy library docs
https://spotipy.readthedocs.io/en/2.19.0/#

"""

#Dependencies
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np

#Importing Spotify client_ID and secret_code
from keys import client_id, client_secret

In [5]:
#adding empty columns to the 'top_100_scrapped_df' to indicate search info
top_100_scrapped_df['track_spotify_ID']=""
top_100_scrapped_df['artist_spotify_ID']=""
top_100_scrapped_df['song_duration[ms]']=""
top_100_scrapped_df['song_release_date']=""
top_100_scrapped_df['spotify_popularity']=""
top_100_scrapped_df['album_name'] =""
top_100_scrapped_df['album_type']=""

#Displaying main table
top_100_scrapped_df


,song_ranking,artist_name,song_title,track_spotify_ID,artist_spotify_ID,song_duration[ms],song_release_date,spotify_popularity,album_name,album_type
0,1,Adele,Easy On Me,,,,,,,
1,2,The Kid LAROI & Justin Bieber,Stay,,,,,,,
2,3,Mariah Carey,All I Want For Christmas Is You,,,,,,,
3,4,Brenda Lee,Rockin' Around The Christmas Tree,,,,,,,
4,5,Bobby Helms,Jingle Bell Rock,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
95,96,Jonas Brothers,Who's In Your Head,,,,,,,
96,97,"Yung Bleu, Chris Brown & 2 Chainz",Baddest,,,,,,,
97,98,42 Dugg Featuring Future,Maybach,,,,,,,
98,99,Aventura x Bad Bunny,Volvi,,,,,,,


In [6]:
#Authenticating requests using Client Credential Flow
scope = "user-library-read"
sp = sp.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

## Search function - Extracting song, artist and album data from Spotify

In [7]:
# function to extract data for any track using search
# Note: The function below extracts song_title from the a table with column_header called 'song_title'

from search_function import *

In [8]:
# Iterating search function for each row of the top_100_scrapped_df 
for index,row in top_100_scrapped_df.iterrows():
    top_100_scrapped_df.loc[index,:] = my_search(row)

# creating output csv for main table
top_100_scrapped_df.to_csv('Output_CSV/top_100_scrapped_df.csv', index=False)


Song details not found for Christmas Tree Farm (Old Timey Version)


In [9]:
# displaying top_100_scrapped_df 
top_100_scrapped_df

,song_ranking,artist_name,song_title,track_spotify_ID,artist_spotify_ID,song_duration[ms],song_release_date,spotify_popularity,album_name,album_type
0,1,Adele,Easy On Me,0gplL1WMoJ6iYaPgMCL0gX,4dpARuHxo51G3z768sgnrY,224694,2021-10-14,100,Easy On Me,single
1,2,The Kid LAROI & Justin Bieber,Stay,5HCyWlXZPP0y6Gqq8TgA20,2tIP7SsRs7vjIcLrU85W8J,141805,2021-07-09,98,STAY (with Justin Bieber),single
2,3,Mariah Carey,All I Want For Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,4iHNK0tOyZPYnBU7nGAgpQ,241106,1994-11-01,96,Merry Christmas,album
3,4,Brenda Lee,Rockin' Around The Christmas Tree,2EjXfH91m7f8HiJN1yQg97,4cPHsZM98sKzmV26wlwD2W,126266,1964-10-19,93,Merry Christmas From Brenda Lee,album
4,5,Bobby Helms,Jingle Bell Rock,7vQbuQcyTflfCIOu3Uzzya,38EmEgXkgK51MT2tPY0EoC,130973,1957-12-02,92,Jingle Bell Rock/Captain Santa Claus (And His ...,single
...,...,...,...,...,...,...,...,...,...,...
95,96,Jonas Brothers,Who's In Your Head,5nhW8I46uDE5sc5ouEbzao,7gOdHgIoIKoe4i9Tta6qdD,183536,2021-09-17,76,Who's In Your Head,single
96,97,"Yung Bleu, Chris Brown & 2 Chainz",Baddest,0sr0YMdDTNNr4Lv4VnI3C8,7wlFDEWiM5OoIAt8RSli8b,160000,2021-09-24,65,"Sincerely, Kentrell",album
97,98,42 Dugg Featuring Future,Maybach,2Qt8qG9SWPdtRiaWcPNJRm,45gHcnDnMC15sgx3VL7ROG,199093,2021-05-21,72,Free Dem Boyz,album
98,99,Aventura x Bad Bunny,Volvi,2vmfvSoZBFAt9hhRoEByLi,1qto4hHid1P71emI6Fd8xi,230125,2021-08-03,92,Volví,single


In [10]:
# dropping songs for failed searchs
nan_value = float("NaN")
top_100_cleaned_df = top_100_scrapped_df.replace("", nan_value)
top_100_cleaned_df = top_100_cleaned_df.dropna(subset = ["track_spotify_ID"])

print(f'\nSongs with no search results have been dropped.\nDataFrame contains {len(top_100_cleaned_df)} rows')

top_100_cleaned_df


Songs with no search results have been dropped.
DataFrame contains 99 rows


,song_ranking,artist_name,song_title,track_spotify_ID,artist_spotify_ID,song_duration[ms],song_release_date,spotify_popularity,album_name,album_type
0,1,Adele,Easy On Me,0gplL1WMoJ6iYaPgMCL0gX,4dpARuHxo51G3z768sgnrY,224694.0,2021-10-14,100.0,Easy On Me,single
1,2,The Kid LAROI & Justin Bieber,Stay,5HCyWlXZPP0y6Gqq8TgA20,2tIP7SsRs7vjIcLrU85W8J,141805.0,2021-07-09,98.0,STAY (with Justin Bieber),single
2,3,Mariah Carey,All I Want For Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,4iHNK0tOyZPYnBU7nGAgpQ,241106.0,1994-11-01,96.0,Merry Christmas,album
3,4,Brenda Lee,Rockin' Around The Christmas Tree,2EjXfH91m7f8HiJN1yQg97,4cPHsZM98sKzmV26wlwD2W,126266.0,1964-10-19,93.0,Merry Christmas From Brenda Lee,album
4,5,Bobby Helms,Jingle Bell Rock,7vQbuQcyTflfCIOu3Uzzya,38EmEgXkgK51MT2tPY0EoC,130973.0,1957-12-02,92.0,Jingle Bell Rock/Captain Santa Claus (And His ...,single
...,...,...,...,...,...,...,...,...,...,...
95,96,Jonas Brothers,Who's In Your Head,5nhW8I46uDE5sc5ouEbzao,7gOdHgIoIKoe4i9Tta6qdD,183536.0,2021-09-17,76.0,Who's In Your Head,single
96,97,"Yung Bleu, Chris Brown & 2 Chainz",Baddest,0sr0YMdDTNNr4Lv4VnI3C8,7wlFDEWiM5OoIAt8RSli8b,160000.0,2021-09-24,65.0,"Sincerely, Kentrell",album
97,98,42 Dugg Featuring Future,Maybach,2Qt8qG9SWPdtRiaWcPNJRm,45gHcnDnMC15sgx3VL7ROG,199093.0,2021-05-21,72.0,Free Dem Boyz,album
98,99,Aventura x Bad Bunny,Volvi,2vmfvSoZBFAt9hhRoEByLi,1qto4hHid1P71emI6Fd8xi,230125.0,2021-08-03,92.0,Volví,single


## Creating DataFrames - song_df, artist_df, album_df, concert_df

### Inspecting and cleaning the collected data

In [11]:
# ERD Diagram for reference

In [12]:
# adding song_ID
# Each song is unique, hence will have a unique song ID 
songs_limit = len(top_100_cleaned_df)+1
song_ID = range(1,songs_limit)
top_100_cleaned_df['song_ID'] = song_ID

In [13]:
# Grouping by artist spotify ID
artists = top_100_cleaned_df.groupby('artist_spotify_ID')['artist_name'].count()

print(f'There are a total of top {len(artists)} artists in the top 100 songs \nEach artist will be assigned a unique artist_ID')

There are a total of top 81 artists in the top 100 songs 
Each artist will be assigned a unique artist_ID


In [14]:
# creating artist_df

artist_df = top_100_cleaned_df[['artist_name', 'artist_spotify_ID']]
artist_grouped_df = artist_df.drop_duplicates(['artist_spotify_ID'])

# displaying artist_df
artist_grouped_df

,artist_name,artist_spotify_ID
0,Adele,4dpARuHxo51G3z768sgnrY
1,The Kid LAROI & Justin Bieber,2tIP7SsRs7vjIcLrU85W8J
2,Mariah Carey,4iHNK0tOyZPYnBU7nGAgpQ
3,Brenda Lee,4cPHsZM98sKzmV26wlwD2W
4,Bobby Helms,38EmEgXkgK51MT2tPY0EoC
...,...,...
94,Jessie Murph,2yLzlEFtIS0Q9UkyBZdQA7
95,Jonas Brothers,7gOdHgIoIKoe4i9Tta6qdD
96,"Yung Bleu, Chris Brown & 2 Chainz",7wlFDEWiM5OoIAt8RSli8b
97,42 Dugg Featuring Future,45gHcnDnMC15sgx3VL7ROG


In [15]:
# generating csv file for album_table
artist_grouped_df.to_csv('Output_CSV/artist_df.csv', index = False)

In [16]:
# Grouping by album_name
albums = top_100_cleaned_df.groupby('album_name')['album_name'].count()

print(f'There are a total of top {len(albums)} albums in the top 100 songs \nEach album will be assigned a unique album_ID')

There are a total of top 85 albums in the top 100 songs 
Each album will be assigned a unique album_ID


In [17]:
# creating album_df

album_df = top_100_cleaned_df[['album_name', 'album_type']]
album_grouped_df = album_df.drop_duplicates(['album_name'])

# displaying artist_df
album_grouped_df

,album_name,album_type
0,Easy On Me,single
1,STAY (with Justin Bieber),single
2,Merry Christmas,album
3,Merry Christmas From Brenda Lee,album
4,Jingle Bell Rock/Captain Santa Claus (And His ...,single
...,...,...
94,Always Been You,single
95,Who's In Your Head,single
96,"Sincerely, Kentrell",album
97,Free Dem Boyz,album


In [18]:
# creating output csv for main table
album_grouped_df.to_csv('Output_CSV/album_df.csv', index=False)

In [19]:
# creating song_df
song_df = top_100_cleaned_df[['song_ID', 'song_title', 'album_name','track_spotify_ID', 'artist_spotify_ID', 'song_ranking', 'spotify_popularity', 'song_duration[ms]', 'song_release_date']]

# generating csv file for song_table
song_df.to_csv('Output_CSV/song_df.csv', index = False)

# displaying song_table
song_df

,song_ID,song_title,album_name,track_spotify_ID,artist_spotify_ID,song_ranking,spotify_popularity,song_duration[ms],song_release_date
0,1,Easy On Me,Easy On Me,0gplL1WMoJ6iYaPgMCL0gX,4dpARuHxo51G3z768sgnrY,1,100.0,224694.0,2021-10-14
1,2,Stay,STAY (with Justin Bieber),5HCyWlXZPP0y6Gqq8TgA20,2tIP7SsRs7vjIcLrU85W8J,2,98.0,141805.0,2021-07-09
2,3,All I Want For Christmas Is You,Merry Christmas,0bYg9bo50gSsH3LtXe2SQn,4iHNK0tOyZPYnBU7nGAgpQ,3,96.0,241106.0,1994-11-01
3,4,Rockin' Around The Christmas Tree,Merry Christmas From Brenda Lee,2EjXfH91m7f8HiJN1yQg97,4cPHsZM98sKzmV26wlwD2W,4,93.0,126266.0,1964-10-19
4,5,Jingle Bell Rock,Jingle Bell Rock/Captain Santa Claus (And His ...,7vQbuQcyTflfCIOu3Uzzya,38EmEgXkgK51MT2tPY0EoC,5,92.0,130973.0,1957-12-02
...,...,...,...,...,...,...,...,...,...
95,95,Who's In Your Head,Who's In Your Head,5nhW8I46uDE5sc5ouEbzao,7gOdHgIoIKoe4i9Tta6qdD,96,76.0,183536.0,2021-09-17
96,96,Baddest,"Sincerely, Kentrell",0sr0YMdDTNNr4Lv4VnI3C8,7wlFDEWiM5OoIAt8RSli8b,97,65.0,160000.0,2021-09-24
97,97,Maybach,Free Dem Boyz,2Qt8qG9SWPdtRiaWcPNJRm,45gHcnDnMC15sgx3VL7ROG,98,72.0,199093.0,2021-05-21
98,98,Volvi,Volví,2vmfvSoZBFAt9hhRoEByLi,1qto4hHid1P71emI6Fd8xi,99,92.0,230125.0,2021-08-03


# Concert Data

In [20]:
import json
from api_keys import BIT_api
import pprint
import os

In [21]:
# converting artinst_names to list
artist_list = artist_grouped_df['artist_name'].tolist()

# printing the list
artist_list

['Adele',
 'The Kid LAROI & Justin Bieber',
 'Mariah Carey',
 'Brenda Lee',
 'Bobby Helms',
 'Lil Nas X & Jack Harlow',
 'Burl Ives',
 'Glass Animals',
 'Ed Sheeran',
 'Doja Cat',
 'Andy Williams',
 'Elton John & Dua Lipa',
 'Silk Sonic (Bruno Mars & Anderson .Paak)',
 'Wham!',
 'Jose Feliciano',
 'Nat King Cole',
 'The Ronettes',
 'Michael Buble',
 'Taylor Swift',
 'Dean Martin',
 'Walker Hayes',
 'GAYLE',
 'Bing Crosby',
 'Olivia Rodrigo',
 'Kelly Clarkson',
 'Wizkid Featuring Justin Bieber & Tems',
 'Drake Featuring Future & Young Thug',
 'Maneskin',
 'Post Malone & The Weeknd',
 'Ariana Grande',
 'The Weeknd & Ariana Grande',
 'Justin Bieber',
 'Chuck Berry',
 'Jason Aldean & Carrie Underwood',
 'Billie Eilish',
 'NEIKED X Mae Muller X Polo G',
 'CKay',
 'THE ANXIETY: WILLOW & Tyler Cole',
 'Dustin Lynch Featuring Lauren Alaina Or MacKenzie Porter',
 'Coldplay x BTS',
 'Luke Combs',
 'Nardo Wick Featuring G Herbo, Lil Durk & 21 Savage',
 'Jordan Davis Featuring Luke Bryan',
 'Kodak

In [22]:
# importing event_api function from events_api.py
from events_api import *

In [23]:
# generating event_df
initial_event_df = event_api(artist_list)

initial_event_df

,artist_name,api_artist,venue,country,location,datetime
0,Moneybagg Yo,MoneyBagg Yo,Little Caesars Arena,United States,"Detroit, MI",2021-12-17
1,Adele,Adele,The Colosseum at Caesars Palace,United States,"Las Vegas, NV",2022-01-21
2,NEIKED X Mae Muller X Polo G,Not Found,Not Found,Not Found,Not Found,Not Found
3,Silk Sonic (Bruno Mars & Anderson .Paak),Not Found,Not Found,Not Found,Not Found,Not Found
4,Summer Walker & SZA,Not Found,Not Found,Not Found,Not Found,Not Found
...,...,...,...,...,...,...
76,Baby Keem & Kendrick Lamar,Not Found,Not Found,Not Found,Not Found,Not Found
77,Coldplay x BTS,Not Found,Not Found,Not Found,Not Found,Not Found
78,Wizkid Featuring Justin Bieber & Tems,Not Found,Not Found,Not Found,Not Found,Not Found
79,Morgan Wallen,Morgan Wallen,Ford Center,United States,"Evansville, IN",2022-02-03


In [24]:
#adding 'artist_spotify_ID' column
merged_df = initial_event_df.merge(top_100_cleaned_df, on ='artist_name', how = 'inner')
event_df = merged_df[['artist_spotify_ID', 'api_artist', 'venue', 'country','location','datetime']]
# printing event_df
event_df

,artist_spotify_ID,api_artist,venue,country,location,datetime
0,3tJoFztHeIJkJWMrx0td2f,MoneyBagg Yo,Little Caesars Arena,United States,"Detroit, MI",2021-12-17
1,4dpARuHxo51G3z768sgnrY,Adele,The Colosseum at Caesars Palace,United States,"Las Vegas, NV",2022-01-21
2,4dpARuHxo51G3z768sgnrY,Adele,The Colosseum at Caesars Palace,United States,"Las Vegas, NV",2022-01-21
3,4dpARuHxo51G3z768sgnrY,Adele,The Colosseum at Caesars Palace,United States,"Las Vegas, NV",2022-01-21
4,4dpARuHxo51G3z768sgnrY,Adele,The Colosseum at Caesars Palace,United States,"Las Vegas, NV",2022-01-21
...,...,...,...,...,...,...
84,5SXuuuRpukkTvsLuUknva1,Not Found,Not Found,Not Found,Not Found,Not Found
85,4gzpq5DPGxSnKTe4SA8HAU,Not Found,Not Found,Not Found,Not Found,Not Found
86,3tVQdUvClmAT7URs9V3rsp,Not Found,Not Found,Not Found,Not Found,Not Found
87,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,Ford Center,United States,"Evansville, IN",2022-02-03


In [35]:
# generating csv file for concert_table
event_df.to_csv('Output_CSV/event_df.csv')

# displaying album_table
event_df = event_df.drop_duplicates()
event_df

,artist_spotify_ID,api_artist,venue,country,location,datetime
0,3tJoFztHeIJkJWMrx0td2f,MoneyBagg Yo,Little Caesars Arena,United States,"Detroit, MI",2021-12-17
1,4dpARuHxo51G3z768sgnrY,Adele,The Colosseum at Caesars Palace,United States,"Las Vegas, NV",2022-01-21
6,5H6xmHXjsq98NLbEjuE29f,Not Found,Not Found,Not Found,Not Found,Not Found
7,0du5cEVh5yTK9QJze8zA0C,Not Found,Not Found,Not Found,Not Found,Not Found
8,57LYzLEk2LcFghVwuWbcuS,Not Found,Not Found,Not Found,Not Found,Not Found
...,...,...,...,...,...,...
84,5SXuuuRpukkTvsLuUknva1,Not Found,Not Found,Not Found,Not Found,Not Found
85,4gzpq5DPGxSnKTe4SA8HAU,Not Found,Not Found,Not Found,Not Found,Not Found
86,3tVQdUvClmAT7URs9V3rsp,Not Found,Not Found,Not Found,Not Found,Not Found
87,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,Ford Center,United States,"Evansville, IN",2022-02-03


# SQL Alchemy ORM 

In [26]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from urllib.parse import quote_plus as urlquote
from credentials import *
import datetime as dt 

In [27]:
connection_url = URL.create(
    drivername = "postgresql", 
    username = db_user,
    password = db_password,
    host = "localhost", 
    port = 5432,
    database = "music_ddl_create_table", 
)

engine = create_engine(connection_url)

/Users/shrutihindocha/anaconda3/envs/PythonData/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [28]:
# Reflect postgresql database
from sqlalchemy import MetaData
metadata_obj = MetaData()
metadata_obj.reflect(bind=engine)
artist = metadata_obj.tables["artist"]
songs = metadata_obj.tables["songs"]
album = metadata_obj.tables["album"]
concert = metadata_obj.tables["concert"]

In [29]:
# Upsert: artist
insert_statement = postgresql.insert(artist).values(artist_grouped_df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['artist_spotify_ID'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['artist_spotify_ID']})
engine.execute(upsert_statement)

In [30]:
# Upsert: songs
insert_statement = postgresql.insert(songs).values(song_df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['song_ID'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['song_ID']})
engine.execute(upsert_statement)


In [31]:
# Upsert album
insert_statement = postgresql.insert(album).values(album_grouped_df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['album_name'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['album_name']})
engine.execute(upsert_statement)


In [36]:
# Upsert concert
insert_statement = postgresql.insert(concert).values(event_df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['artist_spotify_ID'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['artist_spotify_ID']})
engine.execute(upsert_statement)

In [37]:
print(f"ETL job completed at {dt.datetime.now()}")

ETL job completed at 2021-12-13 22:05:27.122494
